In [1]:
# Imports
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import os
import shapefile
from geopy.distance import geodesic

# !pip install geocoder
from geopy.geocoders import Nominatim

In [2]:
# Import functions
# Function that calculates the distance in km between two points using the latitude and longitude data 
def get_lat_lon_dist(row):
    
    latlon1 = tuple(row[['latitude1', 'longitude1']])
    latlon2 = tuple(row[['latitude2', 'longitude2']])

    return geodesic(latlon1, latlon2).kilometers



def compute_distances(df_pois, data_entities):

    # In order to be able to apply the function defined above with the data from the file with the points of interest, 
    data_entities[['latitude', 'longitude']] = data_entities[["Lat", "Long"]]
    df_pois[['latitude', 'longitude']] = df_pois[["Latitude", "Longitude"]]
    

    # Cross-join to get all combinations of latitude/longitude
    dist = pd.merge(data_entities.copy().assign(k=1), df_pois.copy().assign(k=1), on='k', suffixes=('1', '2')).drop('k', axis=1)
    
    # Application of the get_lat_lon_dist function defined with the data from the points of interest and the sensors. 
    # Creation of a new column "dist_NOME_DA_ENTIDADE" with the distance in km from each sensor to each point of interest
    dist['dist_NOME_DA_ENTIDADE'] = dist.apply(get_lat_lon_dist, axis=1)
    
    return dist


def return_amount_of_points_of_interest_per_sensor(distances_df, threshold):
    
    # We set a thresold distance equal to 1.5km because we consider, that given the size of the Porto region, 1.5km is a walkable distance. 
    # For each sensor, we defined the number of restaurants, hotels, shopping centers, etc that are at a distance of 1.5km or less
    dist_new = distances_df.copy()
    dist_new['is_below_threshold'] = np.where(dist_new['dist_NOME_DA_ENTIDADE'] <= threshold, 1, 0)
    
    
    # Check this later
    # sensor_categ = dist_new.groupby(['entity_id', 'category'])['is_below_threshold'].sum().reset_index()
    # sensor_categ = sensor_categ.pivot_table(index = "entity_id", columns = "category", values = "is_below_threshold")
    
    return dist_new

In [3]:
# Get the Points of Interest of this Region
# We start by loading the .CSV file
businesses = pd.read_csv('data/businesses.csv', delimiter=';')

# Let's show 
businesses.head()

,WKT,ADDRESS,OPEN YEAR,OPEN MONTH,TYPE,Description,Merchandise Type
0,POINT (1396322.217 4990301.69),VIA CLAUDIO LUIGI BERTHOLLET 24,1977,1,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari
1,POINT (1396322.217 4990301.69),VIA CLAUDIO LUIGI BERTHOLLET 24,1985,6,ALIMENTARI,PICCOLE STRUTTURE,Panificio
2,POINT (1396303.762 4990325.001),VIA CLAUDIO LUIGI BERTHOLLET 25/F,2017,9,ALTRO,DIA di somministrazione,Nessuna
3,POINT (1396434.395 4990540.6),CORSO VITTORIO EMANUELE II 21/A,2013,10,ALTRO,DIA di somministrazione,Nessuna
4,POINT (1396434.395 4990540.6),CORSO VITTORIO EMANUELE II 21/A,2009,2,ALTRO,DIA di somministrazione,Nessuna


In [ ]:
# To obtain the coordinates we performed a reversed mapping of the address
# Create a Geolocator
geolocator = Nominatim(user_agent="wdl-tech-moguls")

# Iterate through the businesses dataframe
for i in range(len(businesses)):
    location = geolocator.geocode(businesses.loc[i, "ADDRESS"])
    businesses.loc[i, "Longitude"], businesses.loc[i, "Latitude"] = location.longitude, location.latitude


# We saved this into a .CSV for further use
businesses.to_csv("data/businesses_proc.csv")

In [4]:
# Let's check if it is everything OK
businesses = pd.read_csv("data/businesses_proc.csv")
businesses

,Unnamed: 0,WKT,ADDRESS,OPEN YEAR,OPEN MONTH,TYPE,Description,Merchandise Type,Longitude,Latitude
0,0,POINT (1396322.217 4990301.69),VIA CLAUDIO LUIGI BERTHOLLET 24,1977,1,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,7.682778,45.058444
1,1,POINT (1396322.217 4990301.69),VIA CLAUDIO LUIGI BERTHOLLET 24,1985,6,ALIMENTARI,PICCOLE STRUTTURE,Panificio,7.682778,45.058444
2,2,POINT (1396303.762 4990325.001),VIA CLAUDIO LUIGI BERTHOLLET 25/F,2017,9,ALTRO,DIA di somministrazione,Nessuna,7.682632,45.058607
3,3,POINT (1396434.395 4990540.6),CORSO VITTORIO EMANUELE II 21/A,2013,10,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567
4,4,POINT (1396434.395 4990540.6),CORSO VITTORIO EMANUELE II 21/A,2009,2,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567
...,...,...,...,...,...,...,...,...,...,...
823,823,POINT (1396561.148 4990473.514),CORSO VITTORIO EMANUELE II 9,2016,9,ALIMENTARI,PICCOLE STRUTTURE,217,8.401330,40.837933
824,824,POINT (1396561.148 4990473.514),CORSO VITTORIO EMANUELE II 9,2016,7,EXTRALIMENTARI,PICCOLE STRUTTURE,208,8.401330,40.837933
825,825,POINT (1396561.148 4990473.514),CORSO VITTORIO EMANUELE II 9,1993,4,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,8.401330,40.837933
826,826,POINT (1396319.554 4990303.098),VIA CLAUDIO LUIGI BERTHOLLET 24/C,2001,12,EXTRALIMENTARI,PICCOLE STRUTTURE,Abbigliamento,7.682837,45.058413


In [5]:
# Load list of sensors
sensor_list = pd.read_csv('data/noise_sensor_list.csv', sep = ';')
sensor_list['Sensor_ID'] = ['C1', 'C2', 'C3', 'C4', 'C5']
sensor_list['Lat'] = sensor_list['Lat'].str.replace(',', '.').astype(float)
sensor_list['Long'] = sensor_list['Long'].str.replace(',', '.').astype(float)

# Get mapping locations and correspondence to area type
# Link: https://webgis.arpa.piemonte.it/Geoviewer2D/?config=other-configs/acustica_config.json

mapping_location_area_code = pd.DataFrame(
    [['s_01', 65, 55, 'IV - Aree di intensa attività umana'],
    ['s_02', 60, 50, 'III - Aree di tipo misto'],
    ['s_03', 60, 50, 'III - Aree di tipo misto'],
    ['s_05', 65, 55, 'IV - Aree di intensa attività umana'],
    ['s_06', 60, 50, 'III - Aree di tipo misto']],
    columns=['code', 'day_max_db', 'night_max_db', 'area_type']
)

sensor_list = sensor_list.merge(mapping_location_area_code, on=['code'])

In [6]:
# Let's now compute the distances between each point of interest and each sensor
distances = compute_distances(businesses, sensor_list)
distances

,code,address,Lat,Long,streaming,Sensor_ID,day_max_db,night_max_db,area_type,latitude1,...,OPEN YEAR,OPEN MONTH,TYPE,Description,Merchandise Type,Longitude,Latitude,latitude2,longitude2,dist_NOME_DA_ENTIDADE
0,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,45.059172,...,1977,1,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,7.682778,45.058444,45.058444,7.682778,0.309472
1,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,45.059172,...,1985,6,ALIMENTARI,PICCOLE STRUTTURE,Panificio,7.682778,45.058444,45.058444,7.682778,0.309472
2,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,45.059172,...,2017,9,ALTRO,DIA di somministrazione,Nessuna,7.682632,45.058607,45.058607,7.682632,0.293933
3,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,45.059172,...,2013,10,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567,45.060567,7.684246,0.442336
4,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,45.059172,...,2009,2,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567,45.060567,7.684246,0.442336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,45.055554,...,2016,9,ALIMENTARI,PICCOLE STRUTTURE,217,8.401330,40.837933,40.837933,8.401330,472.194362
4136,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,45.055554,...,2016,7,EXTRALIMENTARI,PICCOLE STRUTTURE,208,8.401330,40.837933,40.837933,8.401330,472.194362
4137,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,45.055554,...,1993,4,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,8.401330,40.837933,40.837933,8.401330,472.194362
4138,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,45.055554,...,2001,12,EXTRALIMENTARI,PICCOLE STRUTTURE,Abbigliamento,7.682837,45.058413,45.058413,7.682837,0.318278


In [7]:
# We save it into CSV for further use
distances.to_csv("data/distances_sensors_pois.csv")

In [8]:
# Check if everything is OK
distances = pd.read_csv("data/distances_sensors_pois.csv")
distances

,Unnamed: 0,code,address,Lat,Long,streaming,Sensor_ID,day_max_db,night_max_db,area_type,...,OPEN YEAR,OPEN MONTH,TYPE,Description,Merchandise Type,Longitude,Latitude,latitude2,longitude2,dist_NOME_DA_ENTIDADE
0,0,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,...,1977,1,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,7.682778,45.058444,45.058444,7.682778,0.309472
1,1,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,...,1985,6,ALIMENTARI,PICCOLE STRUTTURE,Panificio,7.682778,45.058444,45.058444,7.682778,0.309472
2,2,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,...,2017,9,ALTRO,DIA di somministrazione,Nessuna,7.682632,45.058607,45.058607,7.682632,0.293933
3,3,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,...,2013,10,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567,45.060567,7.684246,0.442336
4,4,s_01,"Via Saluzzo, 26 Torino",45.059172,7.678986,https://userportal.smartdatanet.it/userportal/...,C1,65,55,IV - Aree di intensa attività umana,...,2009,2,ALTRO,DIA di somministrazione,Nessuna,7.684246,45.060567,45.060567,7.684246,0.442336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4135,4135,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,...,2016,9,ALIMENTARI,PICCOLE STRUTTURE,217,8.401330,40.837933,40.837933,8.401330,472.194362
4136,4136,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,...,2016,7,EXTRALIMENTARI,PICCOLE STRUTTURE,208,8.401330,40.837933,40.837933,8.401330,472.194362
4137,4137,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,...,1993,4,EXTRALIMENTARI,PICCOLE STRUTTURE,Extralimentari,8.401330,40.837933,40.837933,8.401330,472.194362
4138,4138,s_06,"Corso Marconi, 27 Torino",45.055554,7.682590,https://userportal.smartdatanet.it/userportal/...,C5,60,50,III - Aree di tipo misto,...,2001,12,EXTRALIMENTARI,PICCOLE STRUTTURE,Abbigliamento,7.682837,45.058413,45.058413,7.682837,0.318278


In [9]:
# Let's now create some variables that will be used in our loop of thresholds
# These will be the variables we intend to keep
cols_to_keep = [
    'code', 
    'address', 
    'Sensor_ID', 
    'day_max_db_x', 
    'night_max_db_x', 
    'area_type_x', 
    'TYPE', 
    'Description', 
    'Merchandise Type', 
    'dist_NOME_DA_ENTIDADE', 
    'is_below_threshold'
]

# These will be the variables we will remove: 
# We have some merchadises types that are not informative (they are given by numbers)
cols_to_remove = ['205', '207', '208', '214', '217', '99']

In [10]:
# Get different dataframes per threshold (in km)
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5]:
    
    # Merge on Sensors
    thresh_df = return_amount_of_points_of_interest_per_sensor(distances, thresh)
    thresh_w_mapping = thresh_df.merge(mapping_location_area_code, on=['code'])
    thresh_w_mapping = thresh_w_mapping[cols_to_keep]
    
    # Count Points of Interest
    sensor_categ = thresh_w_mapping.copy().groupby(['Sensor_ID', 'Merchandise Type'])['is_below_threshold'].sum().reset_index()
    sensor_categ = sensor_categ.copy().pivot_table(index = "Sensor_ID", columns = "Merchandise Type", values = "is_below_threshold")
    sensor_categ = sensor_categ.copy().drop(cols_to_remove, axis = 1).reset_index()
    sensor_categ
    
    # Merge again to have the remaining columns ("day_max_db", "night_max_db", "area_type")
    final_df = sensor_categ.merge(sensor_list.copy()[["Sensor_ID", "day_max_db", "night_max_db", "area_type"]], on=["Sensor_ID"])
    final_df.to_csv(f"data/thresh_df_{thresh}.csv")
    
# Shown the last example
final_df

,Sensor_ID,Abbigliamento,Alimentari,Alimentari annessi ad altra attivita,Articoli per animali,Articoli pr la casa,Articoli sanitari e ortopedici,Audiovisivi,Autoveicoli e motoveicoli,Bibite,...,Ricambi auto e accessori,Sexy shop,Tabacchi,Telefonia,Tessuti,Vendita al dettaglio di cose antiche ed usate,Vendita non esclusiva di giornali,day_max_db,night_max_db,area_type
0,C1,37,55,5,3,3,1,11,4,22,...,1,1,8,15,1,13,1,65,55,IV - Aree di intensa attività umana
1,C2,37,55,5,3,3,1,11,4,22,...,1,1,8,15,1,13,1,60,50,III - Aree di tipo misto
2,C3,37,55,5,3,3,1,11,4,22,...,1,1,8,15,1,13,1,60,50,III - Aree di tipo misto
3,C4,37,55,5,3,3,1,11,4,22,...,1,1,8,15,1,13,1,65,55,IV - Aree di intensa attività umana
4,C5,37,55,5,3,3,1,11,4,22,...,1,1,8,15,1,13,1,60,50,III - Aree di tipo misto


In [11]:
# Check the columns
final_df.columns

Index(['Sensor_ID', 'Abbigliamento', 'Alimentari',
       'Alimentari annessi ad altra attivita', 'Articoli per animali',
       'Articoli pr la casa', 'Articoli sanitari e ortopedici', 'Audiovisivi',
       'Autoveicoli e motoveicoli', 'Bibite', 'Bigiotteria', 'Calzature',
       'Carburanti', 'Cartolerie', 'Casalinghi - Igiene casa e persona',
       'Elettrodomestici e/o ricambi', 'Elettronica', 'Enoteca',
       'Erboristeria', 'Extralimentari', 'Farmacia', 'Ferramenta',
       'Fiori e piante', 'Frutta e verdura', 'Gastronomia', 'Gelateria',
       'Giocattoli', 'Intimo', 'Librerie', 'Macelleria', 'Minimercato',
       'Mista', 'Mobili', 'Negozio con apparecchi automatici', 'Nessuna',
       'Non alimentari annessi ad altre attivita', 'Non alimentari generici',
       'Oggetti preziosi', 'Opere d'arte', 'Ottica', 'Panetteria', 'Panificio',
       'Pasticceria', 'Pescheria', 'Phone center', 'Pizza al taglio',
       'Profumeria', 'Quotidiani e periodici', 'Ricambi auto e accessori'